In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import yaml
import seaborn as sns
import tensorflow as tf
import autokeras as ak #needed to have the definition of specific layers

import numpy as np
from dti_util import tile2im, get_score_importances, decode_dam, code_dam, unstack_training, clean_ax, rmse, corr
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model

In [ ]:
# Change default matplotlib
mpl.rcParams['savefig.dpi']=150
mpl.rcParams['savefig.bbox']='tight'
mpl.rcParams['savefig.pad_inches']=0.05
mpl.rcParams['axes.labelsize']=mpl.rcParams['axes.titlesize']

In [ ]:
# Name of the experiment
#name = 'exp4'
name = 'exp-smooth'
suff ='_smooth'
#suff = ''
# Root of all directory used
rootdir = '/mnt/sfe-ns9602k/Julien/data'

#figdir
figdir = '../overleaf/figs/'

# Directory of the experiment outputs
expdir = os.path.join(rootdir, name)

# Load experiment parameters
with open(os.path.join(expdir,'data_params.yml' )) as file:
    exp_dict = yaml.load(file, Loader=yaml.FullLoader)
    
# Print experiments parameters
for key, value in exp_dict.items():
    print(key, ' : ', value)
    
# Set the used parameters
traindir = exp_dict['traindir']
epsi = exp_dict['epsi']
th_dam = exp_dict['th_dam']
th_sit = exp_dict['th_sit']
dsize = exp_dict['dsize']
colnames = exp_dict['colnames']

In [ ]:
# Normalization function
norm = lambda x : code_dam(x,epsi=epsi, vmin=th_dam)

# Denormalization function
denorm = lambda x : decode_dam(x,epsi=epsi, vmin=th_dam)

lims = {
    'log_deformation_0': (-4, 1),
    'log_deformation_1': (-4, 1),
    'h':(th_sit,5),
    'c':(0.9,1),
    'damage_n' : [norm(1),norm(th_dam)],
    'damage' : [th_dam, 1]

}

In [ ]:
data_test = np.load(os.path.join(traindir,'test.npz'))

In [ ]:
Xtest = data_test['Xtest']
ytest = denorm(data_test['ytest'])
mask_test = data_test['mask_test']
shape_original = data_test['shape_original']
ntest, ny, nx = shape_original
print (f'Size of the output image: {ny}x{nx}')
X2, y2_test = unstack_training(Xtest, ytest, mask_test, ny=ny, nx=nx, strides=1)


In [ ]:
X2.shape

## 1. Feature example

In [ ]:
# Plot test input feature
for i, name in enumerate(colnames):

    fig, ax = plt.subplots(figsize=(5,5))
    im = ax.imshow(X2[0,...,i],vmin=lims[name][0],vmax=lims[name][1])
    #fig.colorbar(im)
    clean_ax([ax])
    fig.savefig(os.path.join(figdir, f'imtest_{name}{suff}.png'))

In [ ]:
# Plot test damage
fig, ax = plt.subplots(figsize=(5,5))
dam = ax.imshow(y2_test[0],vmin=lims['damage'][0], vmax = lims['damage'][1],cmap='jet')
clean_ax([ax])

fig.savefig(os.path.join(figdir, f'imtest_damage.png'))

## 2 Baseline and scores

In [ ]:

# Name of the model
model_name = "long"
#model_name = "demo"

# Read model setting parameter from file
from_file = True

# pretrained model ( if False, launch a new training)
pretrained = True

# Directory to stores the model
model_dir = os.path.join(expdir, model_name)

# Create the model directory if necessary
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)

# Dictionnary sotring all the mod setting parameters
dmod = dict()

if from_file:
    # Read model parameters from yml file
    with open(os.path.join(model_dir,'model_params.yml' )) as file:
        dmod = yaml.load(file, Loader=yaml.FullLoader)        
else:
    # Create a yml file

    # relative size of the validation dataset
    dmod['test_size'] = 0.15

    # Seed of the train/val random split
    dmod['split_seed'] = 1

    # log_dir for tensorboard
    # log_dir is hardcoded because of NIRD toolkit, how to get the log name automatically?
    dmod['log_dir'] = "/mnt/sfe-ns9602k/.tools/deep-learn-1603280065-tensorboard/autokeras/{}-{}".format(name, model_name)

    # Shuffle score file name 
    dmod['fname_score_shuffle'] = 'shuffle_score'

    # Saliency score file name 
    dmod['fname_score_saliency'] = 'saliency_score'

    # Patience of the early stopping
    dmod['patience'] = 15 #15 #5

    # number of epochs
    dmod['epochs'] = 300

    # Number of trials of the model experiment
    dmod['max_trials'] = 50 #50 #3

    # Size of the training set (None for selecting all the training set)
    dmod['ntrain']= None #None #10000

    # Save the dictionary in yaml format
    with open(os.path.join(model_dir,'model_params.yml'),'w') as file:
        yaml.dump(dmod, file)

        
print ('--- MODEL CONFIGURATION ---')
for key, value in dmod.items():
    print(key, ' : ', value)

In [ ]:
data = np.load(os.path.join(traindir,'train.npz'))
X = data['Xtrain']
y = data['ytrain']
mask_train = data['mask_train']
yd = denorm(y)

In [ ]:
print(f"Number of training samples: {X.shape[0]}")
print(f'Size of input feature: {X.shape[1:]}')

In [ ]:
Xtrain, Xval, ytrain, yval = train_test_split(X.astype(np.float32), y.astype(np.float32),
                                                    test_size = dmod['test_size'],
                                                    random_state = dmod['split_seed'])

print(f"Number of training samples: {Xtrain.shape[0]}")
print(f"Number of validation samples: {Xval.shape[0]}")

In [ ]:
def reg_ak(dd):
    return np.clip(1.01 - 0.008/(10**dd)**0.3,0,1)

xpoint = np.array([-3, -2.5, -2, -1.5, -1,  0])
ypoint = np.array([.72, .25, -0.2, -.5, -0.7, -0.9])
coef = np.polyfit(xpoint, ypoint, 2)
print (f'Coefficients of the polynomial fit: {coef}')

def reg_jb(x):
    y = np.poly1d(coef)(x)
    lim = -coef[1]/(2*coef[0])
    y[x>lim] = np.poly1d(coef)(lim)
    y[x<-3.3] = np.poly1d(coef)(-3.3)
    return denorm(y)


In [ ]:
xlin = np.linspace(lims['log_deformation_0'][0],lims['log_deformation_0'][1],100)

y_ak_lin = reg_ak(xlin)
y_jb_lin = reg_jb(xlin)


In [ ]:
fig, ax = plt.subplots()
xx = Xtrain[:,dsize//2,dsize//2,0].ravel()
yy = denorm(ytrain).ravel()

ax.hist2d(xx,yy,  50, [lims['log_deformation_0'],lims['damage'] ], norm=mpl.colors.LogNorm(), cmap='jet');
ax.set_xlabel('Deformation (log)')
ax.set_ylabel('Damage (denormalized)');
fig.savefig(os.path.join(figdir, f'hist2.png'))

ax.plot(xpoint,denorm(ypoint),'+k')
ax.plot(xlin,y_jb_lin,'-k',label="jb baseline")
ax.plot(xlin,y_ak_lin,':k',label="ak baseline")

ax.legend(loc='lower right')
fig.savefig(os.path.join(figdir, f'hist2_baseline{suff}.png'))

In [ ]:
y_ak = reg_ak(Xval[:,dsize//2,dsize//2,0])
y_jb = reg_jb(Xval[:,dsize//2,dsize//2,0])
yval_d = denorm(yval)

In [ ]:
print(os.path.join(model_dir,model_name))

In [ ]:
model = load_model(os.path.join(model_dir,model_name))
ypredict = denorm(model.predict(Xval))


In [ ]:
model.summary()

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
xylims = lims['damage']


ax.hist2d(yval_d.ravel(),y_ak.ravel(),  50, [xylims, xylims], norm=mpl.colors.LogNorm()),
ax.plot(xylims,xylims,'k',linewidth=2)
#ax.set_title('Anton (AK)')
fig.savefig(os.path.join(figdir, f'scatter_ak{suff}.png'))

fig, ax = plt.subplots(figsize=(6,6))

ax.hist2d(yval_d.ravel(),y_jb.ravel(),  50, [xylims, xylims], norm=mpl.colors.LogNorm()),
ax.plot(xylims,xylims,'k',linewidth=2)
#ax[2].set_title('Julien (JB)');
fig.savefig(os.path.join(figdir, f'scatter_jb{suff}.png'))

fig, ax = plt.subplots(figsize=(6,6))

ax.hist2d(yval_d.ravel(),ypredict.ravel(),  50, [xylims, xylims], norm=mpl.colors.LogNorm()),
ax.plot(xylims,xylims,'k',linewidth=2)
#ax[2].set_title('Julien (JB)');
fig.savefig(os.path.join(figdir, f'scatter_nn{suff}.png'))


In [ ]:
ctrue = yval_d > .95
cak = y_ak>.95
cjb = y_jb>.95
cnn = ypredict>.95

In [ ]:
print(classification_report(ctrue, cak))

In [ ]:
print(classification_report(ctrue, cjb))

In [ ]:
print(classification_report(ctrue, cnn))

In [ ]:
# Compute and print correlation
mask= None 

corr_nn = corr(yval_d[mask].ravel(), ypredict[mask].ravel())
corr_ak = corr(yval_d[mask].ravel(), y_ak[mask].ravel())
corr_jb = corr(yval_d[mask].ravel(), y_jb[mask].ravel())
print(f'corr NN: {corr_nn:.2f}\ncorr ak: {corr_ak:.2f}\ncorr jb: {corr_jb:.2f}')

## 3 Inputs features

In [ ]:
for i, name in enumerate(colnames):

    fig, ax = plt.subplots(figsize=(5,5))
    im = ax.imshow(Xval[10,...,i],vmin=lims[name][0],vmax=lims[name][1])
    rect=mpl.patches.Rectangle((11.5, 11.5), 1, 1, linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
    ax.text(12,26,'$x_c$',fontsize='x-large',ha='center')
    ax.text(-2.5,12,'$y_c$',fontsize='x-large',va='center')
    ax.plot([-0.5,12],[12,12],':',color='red')
    ax.plot([12,12],[12,24.5],':',color='red')
    fig.savefig(os.path.join(figdir, f'patch_{name}{suff}.png'))

## 4. test image

In [ ]:
try:
    del Xtrain, ytrain, X, y
except:
    pass
data_test = np.load(os.path.join(traindir,'test.npz'))

In [ ]:
Xtest.shape

In [ ]:
Xval.shape

In [ ]:
ypred_test = denorm(model.predict(Xtest))
y_ak_test = reg_ak(Xtest[:,dsize//2,dsize//2,0])
y_jb_test = reg_jb(Xtest[:,dsize//2,dsize//2,0])

In [ ]:
X2, y2_pred = unstack_training(Xtest, ypred_test, mask_test, ny=ny, nx=nx, strides=1)
X2, y2_ak = unstack_training(Xtest, y_ak_test, mask_test, ny=ny, nx=nx, strides=1)
X2, y2_jb = unstack_training(Xtest, y_jb_test, mask_test, ny=ny, nx=nx, strides=1)

y2_test = y2_test.squeeze()
y2_pred = y2_pred.squeeze()
y2_ak = y2_ak.squeeze()
y2_jb = y2_jb.squeeze()

In [ ]:
delta_jb = y2_jb-y2_test
delta_ak = y2_ak-y2_test
delta_pred = y2_pred-y2_test

#Mask values below the threshold
delta_jb[y2_test<th_dam] = np.nan
delta_ak[y2_test<th_dam] = np.nan
delta_pred[y2_test<th_dam] = np.nan


In [ ]:
fig, ax = plt.subplots(ncols=3,figsize=(16,4))
dam = ax[0].imshow(y2_test,vmin=lims['damage'][0], vmax = lims['damage'][1],cmap='jet')
ax[1].imshow(y2_ak,vmin=lims['damage'][0], vmax=lims['damage'][1],cmap='jet')
ax[2].imshow(y2_pred,vmin=lims['damage'][0], vmax=lims['damage'][1],cmap='jet')
fig.colorbar(dam, ax=ax.ravel().tolist(),orientation='horizontal');
clean_ax(ax)
ax[0].set_title('Truth');
ax[1].set_title('Baseline (AK)');
ax[2].set_title('CNN');
fig.savefig(os.path.join(figdir, f'test_image{suff}.png'))

In [ ]:
fig, ax = plt.subplots(ncols=2,figsize=(10,4))
err = ax[0].imshow(delta_ak,vmin=-0.1,vmax=0.1,cmap=plt.get_cmap('bwr'))
ax[1].imshow(delta_pred,vmin=-0.1,vmax=0.1,cmap=plt.get_cmap('bwr'))

fig.colorbar(err, ax=ax.ravel().tolist(),orientation='horizontal');
clean_ax(ax)
ax[0].set_title('Baseline (AK) err');
ax[1].set_title('CNN err');
fig.savefig(os.path.join(figdir, f'test_image_err{suff}.png'))

## 5. Explainibility

In [ ]:
# Define input and output features od the validation set
X_sk = Xval
y_sk = denorm(yval)

# Define the score
def score_nn (X, y):
    ypredict = denorm(model.predict(X))
    return corr(y.ravel(), ypredict.ravel())

# Compute the score of each feature (the computation is done n_iter times corresponding to n_iter different random shuffle)
base_score, score_decreases = get_score_importances(score_nn, 
                                                    X_sk, 
                                                    y_sk, 
                                                    n_iter=10, 
                                                    pre_shuffle=True
                                                   )

# Compute the mean importance of each input feature
score_mean = np.mean(score_decreases, axis=0)
score_std = np.std(score_decreases, axis=0)


In [ ]:
# Plot the importance
y_pos = np.arange(len(colnames)+1)
fig, ax = plt.subplots()

ax.barh(y_pos[1:], base_score-score_mean, xerr=score_std, align='center')
ax.barh(y_pos[0],base_score)
ax.set_yticks(y_pos);
ax.set_yticklabels(('base score',)+colnames)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Correlation')
ax.set_title('Feature importance');
fig.savefig(os.path.join(figdir, f'feature_importance{suff}.png'))

In [ ]:
from tf_keras_vis.saliency import Saliency
saliency = Saliency(model,
                    model_modifier=None,
                    clone=False)

In [ ]:
#Number of sample used to calculate the saliency map
n_sample = 2000

# Define the loss for the gradient computation
def loss(output):
    return tuple(output[i] for i in range(n_sample))

# Compute the saliency map for each input feature (keepdims=True)
saliency_map = saliency(loss, Xval[:n_sample], keepdims=True)

# Compute the average saliency map
saliency_absmean = np.abs(saliency_map).mean(axis=0)

# To be used to normalize the saliency map with the standard deviation
sigma_in = np.sqrt(model.layers[2].variance.numpy())


In [ ]:
nc = len(exp_dict['colnames'])
vmax = .1 #saliency_absmean.max()
fig, ax = plt.subplots(ncols=nc,figsize=(5*nc,5))
for i in range(nc):
    vmax = saliency_absmean[...,i].max()
    im = ax[i].imshow(saliency_absmean[...,i], cmap='jet', vmax = vmax)
    ax[i].set_title(exp_dict['colnames'][i])
    fig.colorbar(im, ax=ax[i],orientation='horizontal');

#fig.colorbar(im, ax=ax.ravel().tolist(),orientation='horizontal');
fig.suptitle ('Mean saliency map');
fig.savefig(os.path.join(figdir, f'saliency{suff}.png'))